In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
path = 'hdfs://orion11:14001/reddit/sampled_reddit'
df = spark.read.json(path)
df.head()

Row(archived=True, author='halftone84', author_cakeday=None, author_flair_css_class=None, author_flair_text=None, body='I read the title and thought of that cheating bitch clown from the glassjaw video', body_html=None, controversiality=0, created=None, created_utc='1309478410', distinguished=None, downs=0, edited='false', gilded=0, id='c22x4bc', link_id='t3_idgji', mod_reports=None, name='t1_c22x4bc', parent_id='t3_idgji', removal_reason=None, replies=None, retrieved_on=1427302517, saved=None, score=1, score_hidden=False, stickied=None, subreddit='AskReddit', subreddit_id='t5_2qh1i', ups=1, user_reports=None)

In [3]:
df.columns

['archived',
 'author',
 'author_cakeday',
 'author_flair_css_class',
 'author_flair_text',
 'body',
 'body_html',
 'controversiality',
 'created',
 'created_utc',
 'distinguished',
 'downs',
 'edited',
 'gilded',
 'id',
 'link_id',
 'mod_reports',
 'name',
 'parent_id',
 'removal_reason',
 'replies',
 'retrieved_on',
 'saved',
 'score',
 'score_hidden',
 'stickied',
 'subreddit',
 'subreddit_id',
 'ups',
 'user_reports']

In [4]:
type(df)

pyspark.sql.dataframe.DataFrame

In [5]:
df.createOrReplaceTempView('reddit_data')

In [6]:
query = '''
SELECT
  subreddit, 
  body
FROM
  reddit_data
WHERE
  body != "[deleted]"
'''
text_all = spark.sql(query)
text_all.show()

+-----------------+--------------------+
|        subreddit|                body|
+-----------------+--------------------+
|        AskReddit|I read the title ...|
|        AskReddit|Because you're ab...|
|         politics|All he had was a ...|
|        AskReddit|Flushing with you...|
|             IAmA|I grew up in Texa...|
|       California|I did this alread...|
|             pics|Ctrl-F: *drr* *dr...|
|       reddit.com|Damn. I was wrong...|
|        AskReddit|          Me too. :/|
|        AskReddit|I don't know, but...|
|           trance|How did you do that?|
|           google|I can haz google+...|
|             IAmA|Months?  That's a...|
|         sandiego|Is that the Reddi...|
|       googleplus|I got my invite, ...|
|       technology|This sounds bette...|
|        worldnews|Israel handled it...|
|DrawingFromWithin|Yeah, I'm ruler g...|
|           gaming|I just found out ...|
|       reddit.com|Hope you can't wa...|
+-----------------+--------------------+
only showing top

In [7]:
# text_all.coalesce(1).write.format('com.databricks.spark.csv').option('header', 'true').save('hdfs://orion11:14001/output/reddit')

In [8]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

import bz2
import json
import sys
import os.path

In [9]:
sid = SentimentIntensityAnalyzer()
# commentCount = 0

def analyze_sentiment(row): 
    
    # line = bz_file.readline().decode('utf8')
#     if len(line) == 0:
#         return
        
#     comment = json.loads(line)
#     print(row)
    subreddit = row["subreddit"]
    body = row["body"]

    # variables to hold the overall average compound score for message
    finalScore = 0
    roundedFinalScore = 0

    # variables to hold the highest positive score in the message
    # and highest negative score in the message
    maxPosScore = 0
    maxNegScore = 0

#     print("===")
    commentLines = tokenize.sent_tokenize(body)
    for line in commentLines:
        ss = sid.polarity_scores(line)
        # uncomment these lines if you want to print out sentences & scores
        '''
        line = line.replace('\n', ' ').replace('\r', '')
        print(line)
        for k in sorted(ss):
            print(' {0}: {1}\n'.format(k,ss[k]), end='')
        '''
        lineCompoundScore = ss['compound']
        finalScore += lineCompoundScore

        if ss['pos'] > maxPosScore:
            maxPosScore = ss['pos']
        elif ss['neg'] > maxNegScore:
            maxNegScore = ss['neg']

    # roundedFinalScore is the average compound score for the entire message
    commentLength = len(commentLines)
    if commentLength == 0:
        commentLength = 1
        
    roundedFinalScore = round(finalScore / commentLength, 4)
    
#     commentCount += 1
#     if commentCount % 1000 == 0:
#         print(commentCount)
#         # break

#     return [str(roundedFinalScore), str(maxPosScore), str(maxNegScore), subreddit]
#     return "{0}\t{1}\t{2}\t{3}\n".format(roundedFinalScore, maxPosScore, maxNegScore, subreddit)   
#     return [float(roundedFinalScore), float(maxPosScore), float(maxNegScore), subreddit]    
#     return (float(roundedFinalScore), subreddit)   
    return (subreddit, float(roundedFinalScore))   


In [10]:
# sentiment_scores = text_all.rdd.flatMap(analyze_sentiment)
text_all.rdd.take(1)
# sentiment_scores.take(100)

[Row(subreddit='AskReddit', body='I read the title and thought of that cheating bitch clown from the glassjaw video')]

In [11]:
# from pyspark.sql.types import StructType, StructField, FloatType, StringType
# # features = [StructField('roundedFinalScore'.strip(), StringType(), True), StructField('maxPosScore'.strip(), StringType(), True), \
# #                      StructField('maxNegScore'.strip(), StringType(), True), StructField('subreddit'.strip(), StringType(), True)]
# features = [StructField('roundedFinalScore'.strip(), FloatType(), True), StructField('maxPosScore'.strip(), FloatType(), True), \
#                      StructField('maxNegScore'.strip(), FloatType(), True), StructField('subreddit'.strip(), StringType(), True)]
# schema = StructType(features)
# print(schema)

In [12]:
def conv(string):
    try:
        return float(string)
    except:
        return 0.0
    
# sentiment_scores = text_all.rdd.flatMap(lambda row: [conv(r) for r in analyze_sentiment(row)]).toDF(schema)
# sentiment_scores = text_all.rdd.flatMap(analyze_sentiment)
sentiment_scores = text_all.rdd.map(analyze_sentiment)
sentiment_scores.take(1)

[('AskReddit', -0.8126)]

In [13]:
sentiment_scores.take(4)

[('AskReddit', -0.8126),
 ('AskReddit', 0.0),
 ('politics', -0.2552),
 ('AskReddit', -0.2384)]

In [14]:
sentiment_scores_sub = sentiment_scores.take(10)
sentiment_scores_sub

[('AskReddit', -0.8126),
 ('AskReddit', 0.0),
 ('politics', -0.2552),
 ('AskReddit', -0.2384),
 ('IAmA', 0.0979),
 ('California', 0.0),
 ('pics', 0.0),
 ('reddit.com', 0.0075),
 ('AskReddit', -0.17),
 ('AskReddit', 0.4465)]

In [15]:
sentiment_scores

PythonRDD[22] at RDD at PythonRDD.scala:53

In [16]:
sentiment_scores_sub

[('AskReddit', -0.8126),
 ('AskReddit', 0.0),
 ('politics', -0.2552),
 ('AskReddit', -0.2384),
 ('IAmA', 0.0979),
 ('California', 0.0),
 ('pics', 0.0),
 ('reddit.com', 0.0075),
 ('AskReddit', -0.17),
 ('AskReddit', 0.4465)]

In [17]:
# TODO: it takes time!
sentiment_scores = sentiment_scores.reduceByKey(lambda x, y: x + y)
sentiment_scores.take(10)

[('LightNovels', 2188.2942999999977),
 ('GermanPractice', 25.700699999999994),
 ('shittyprogramming', 298.91109999999975),
 ('ghibli', 915.9782999999978),
 ('StudentLoans', 767.7833999999998),
 ('techolitics', 232.2476999999995),
 ('ewu', 14.1128),
 ('paranormalromance', 20.2458),
 ('turfing', -0.3125),
 ('PumpUpMusic', 0.5544)]

In [18]:
sentiment_scores.sortBy(lambda x: x[1]).take(5)

[('news', -69372.80710000006),
 ('worldnews', -56732.96110000015),
 ('de', -24307.399600000015),
 ('podemos', -12112.384499999995),
 ('argentina', -11194.748800000001)]

In [19]:
sentiment_scores.sortBy(lambda x: x[1], False).take(5)

[('AskReddit', 972887.5779000001),
 ('leagueoflegends', 387322.6532000008),
 ('pics', 288720.4549999993),
 ('gonewild', 275213.59609999973),
 ('gaming', 262655.0113999998)]

In [22]:
for row in sentiment_scores.collect():
    if row[0] == 'mac':
        print('mac:', row[1])

mac: 4431.5900999999985


In [24]:
for row in sentiment_scores.collect():
    if row[0] == 'windows':
        print('windows:', row[1])

windows: 3270.1883999999986


In [22]:
# rdd = sc.parallelize([('a', 5), ('c', 8), ('b', 6), ('a', 8), ('b', 8), ('a', 3)])
# rdd.reduceByKey(lambda x, y: x + y).collect()

In [27]:
a = [1, 2]
b = [3, 4]
a + b

[1, 2, 3, 4]

In [ ]:
# text_all.write.saveAsTable('text_all')

In [ ]:
text_all.createOrReplaceTempView('text_all')

In [ ]:
query = '''
SELECT
  subreddit, 
  body
FROM
  text_all
GROUP BY subreddit
'''
text_all = spark.sql(query)
text_all.show()

In [ ]:
sentiment_scores = text_all.rdd.flatMap(analyze_sentiment).toDF().show(2)

In [ ]:
dic = {}
for row in text_all.rdd.collect(): 
    subreddit = row['subreddit']
    text = row['body']
    if subreddit in dic:
        dic[subreddit].append(text)
    else:
        texts = []
        texts.append(text)
        dic[subreddit] = texts

In [ ]:
dic = {}

In [ ]:
text = 'bob'
num = 12
if 'bob' in dic: 
    dic[text].append(num)
else: 
    nums = []
    nums.append(num)
    dic[text] = nums

In [ ]:
dic

In [ ]:
num = 15
if 'bob' in dic: 
    dic[text].append(num)
else: 
    nums = []
    nums.append(num)
    dic[text] = nums

In [ ]:
dic